Импортирую библиотеки

In [2]:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

import seaborn as sns 
import matplotlib.pyplot as plt 
sns.set(rc={'figure.figsize':(16,6)}, style='whitegrid')


Подгружаю датасет, смотрю на данные, пустые значения

In [3]:
df = pd.read_csv('cars.csv')

In [4]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [5]:
df.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [6]:
df.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

### 1. Генерирую новый признак. Вместо колонки CarName с полным названием машин, например 'audi 100 ls', создадим колонку только с маркой автомобиля. Ненужные колонки CarName и car_ID удаляю. 

In [110]:
df['company'] = df.CarName.apply(lambda x: x.split(' ')[0])

In [111]:
df.company.nunique()

28

In [112]:
df.drop('car_ID', axis = 'columns', inplace = True)
df.drop('CarName', axis = 'columns', inplace = True)

In [113]:
df.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [114]:
df.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

В колонке company есть ошибки в названиях, Nissan - nissan, toyota - toyouta. Исправим их

In [115]:
# приведем значения колонки company к нижнему регистру

df['company']=df.company.str.lower()

In [116]:
# заменим значения, в названиях которых есть ошибка, на правильные с помощью метода replace(), который обернем в функцию

def fix (what_rep, how_rep):
    df.company.replace(what_rep, how_rep, inplace=True)
    return

In [117]:
# вызываю функцию fix

fix('maxda', 'mazda')
fix('vokswagen', 'volkswagen')
fix('vw', 'volkswagen')
fix('toyouta', 'toyota')
fix('porcshce', 'porsche')

In [118]:
# еще раз проверяю уникальные значения

df.company.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [119]:
df.company.nunique()

22

### 2. Чтобы не перегружать модель большим количеством предикторов, удаляю ненужные. Создаю новый датасет со следующими колонками:
* 'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'

In [123]:
# создаю список с названием только тех колонок, которые мне понадобятся

new_col = ['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price']

In [133]:
# перезаписываю датафрейм с учетом тех колонок, которые мне нужны

df = df[new_col]

In [134]:
df.head()

,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,13495.0
1,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,16500.0
2,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154,16500.0
3,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102,13950.0
4,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115,17450.0


Итого вместо изначальных 25 колонок, в дальнейшем буду работать только с 14

### 3. Строю матрицу корреляция между переменными

In [135]:
df.corr().round(2)

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.00,0.87,0.80,0.78,0.57,0.49,0.35,0.58
carlength,0.87,1.00,0.84,0.88,0.68,0.61,0.55,0.68
carwidth,0.80,0.84,1.00,0.87,0.74,0.56,0.64,0.76
curbweight,0.78,0.88,0.87,1.00,0.85,0.65,0.75,0.84
enginesize,0.57,0.68,0.74,0.85,1.00,0.58,0.81,0.87
boreratio,0.49,0.61,0.56,0.65,0.58,1.00,0.57,0.55
horsepower,0.35,0.55,0.64,0.75,0.81,0.57,1.00,0.81
price,0.58,0.68,0.76,0.84,0.87,0.55,0.81,1.00


### 4. Линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому необходимо применить pd.get_dummies() для перевода категориальных переменных (всего в перезаписанном датасете их 7) в числовой формат.

In [137]:
df.dtypes

company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
price             float64
dtype: object

In [138]:
df.shape

(205, 15)

Для начала перевожу категориальные переменные в числовые, и перезаписываю это в датафрейм df_dummy

In [140]:
df_dummy = pd.get_dummies(data=df[['company','fueltype','aspiration','carbody','drivewheel','enginetype','cylindernumber']],drop_first=True)

Объединяю датафреймы df_dummy и изначальный df(предварительно удалив повторяющиеся колонки)

In [144]:
df = pd.concat([df.drop(['company','fueltype','aspiration','carbody','drivewheel','enginetype','cylindernumber'], axis='columns'), df_dummy],axis=1)

In [148]:
df.head()

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,88.6,168.8,64.1,2548,130,3.47,111,13495.0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,88.6,168.8,64.1,2548,130,3.47,111,16500.0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,94.5,171.2,65.5,2823,152,2.68,154,16500.0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,99.8,176.6,66.2,2337,109,3.19,102,13950.0,1,0,...,1,0,0,0,0,1,0,0,0,0
4,99.4,176.6,66.4,2824,136,3.19,115,17450.0,1,0,...,1,0,0,0,1,0,0,0,0,0


### 5. Cтрою небольшую модель всего с одним предиктором цены (price) – horsepower.

In [159]:
model_1 = smf.ols('price ~ horsepower', df).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Fri, 10 Mar 2023   Prob (F-statistic):           1.48e-48
Time:                        14:48:13   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

### 6. Cтрою новую модель со всеми предикторами и отдельно модель со всеми предикторами, кроме марок машин

#### Модель со всеми предикторами

In [166]:
# для начала сохраняем в отедльную переменную х все предикторы из датасета, за исключением price, 
# т.к price это наша зависимая переменная

X = df.drop(['price'], axis='columns')

# затем добавляем константу, чтобы у нас был intercept

X = sm.add_constant(x)

# в переменную y передаем значение нашей зависимой переменной price

y = df['price']


In [168]:
model_2 = sm.OLS(y,X).fit()
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Fri, 10 Mar 2023   Prob (F-statistic):           4.86e-89
Time:                        15:04:37   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

#### Модель со всеми предикторами, за исключением переменной с марками машин

In [169]:
# находим и убираем из датасета все данные, которые относятся к переменной 'company'(марка машины)

cols_to_keep = df.columns[~df.columns.str.startswith('company_')]
cols_to_keep

Index(['wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize',
       'boreratio', 'horsepower', 'price', 'fueltype_gas', 'aspiration_turbo',
       'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan',
       'carbody_wagon', 'drivewheel_fwd', 'drivewheel_rwd', 'enginetype_dohcv',
       'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf', 'enginetype_ohcv',
       'enginetype_rotor', 'cylindernumber_five', 'cylindernumber_four',
       'cylindernumber_six', 'cylindernumber_three', 'cylindernumber_twelve',
       'cylindernumber_two'],
      dtype='object')

In [182]:
# убираю колонку price т.к это зависимая переменная

X = df[cols_to_keep].drop('price', axis = 'columns')

# затем добавляем константу, чтобы у нас был intercept

X = sm.add_constant(X)

# в переменную y передаем значение нашей зависимой переменной price

y = df['price']

/nfs/env/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [183]:
model_3 = sm.OLS(y, X).fit()
print(model_3.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Fri, 10 Mar 2023   Prob (F-statistic):           9.86e-81
Time:                        15:27:24   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

### Видно, что R2 модели №3 изменился не значимо после удаления переменной company, при этом если обратить внимание на модель №2, можно заметить, что большая часть значений company это мусорные значения, т.к pvalue для этих значений незначимо.

* Вывод 1: Лучше оставить ту модель, где меньше предикторов, ведь R2 изменился не очень сильно, а часть марок вообще не значима
* Вывод 2: Выбранная модель объясняет примерно 90% дисперсии (окр. до целого). Среди предикторов 10 из 27 оказались не значимыми (p > 0.05). Пример интерпретации: при единичном изменении показателя horsepower, цена ВОЗРАСТАЕТ на 86.8164